In [648]:
import PIL
import json
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image,ImageOps,ImageFilter
import keras
from keras import layers
import os
from tensorflow.keras.utils import Sequence
from tensorflow.keras import Sequential
import tensorflow as tf
from keras.preprocessing import image  
import cv2
import pickle

In [649]:
from keras.models import load_model
model_best = load_model('model/self_trained/distracted-25-0.99.hdf5')

In [650]:
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    #img = image.load_img(img_path)
    img = cv2.imread(img_path)
    img = cv2.resize(img, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
    # convert PIL.Image.Image type to 3D tensor with shape (128, 128, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 128, 128, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

In [651]:
infile = open("pickle_files/labels_list.pkl",'rb')
new_dict = pickle.load(infile)
infile.close()
print(new_dict)

{'c6': 0, 'c7': 1, 'c3': 2, 'c1': 3, 'c2': 4, 'c9': 5, 'c0': 6, 'c5': 7, 'c8': 8, 'c4': 9}


In [737]:
from PIL import ImageFile    
from keras.applications.vgg16 import VGG16  
ImageFile.LOAD_TRUNCATED_IMAGES = True     

img_path = 'car_test_9.jpg'
test_tensors = path_to_tensor(img_path).astype('float32')/255 - 0.5

In [738]:
ypred_test = model_best.predict(test_tensors,verbose=1)
ypred_class = np.argmax(ypred_test,axis=1)

1/1 [==============================] - 0s 56ms/step


In [739]:
print(ypred_class)

[7]


In [740]:
infile = open("pickle_files/labels_list.pkl",'rb')
labels_id = pickle.load(infile)
infile.close()

In [741]:
id_labels = dict()
for class_name,idx in labels_id.items():
    id_labels[idx] = class_name
print(id_labels)
ypred_class = int(ypred_class)
res = id_labels[ypred_class]

{0: 'c6', 1: 'c7', 2: 'c3', 3: 'c1', 4: 'c2', 5: 'c9', 6: 'c0', 7: 'c5', 8: 'c8', 9: 'c4'}


In [742]:
class_name = dict()
class_name["c0"] = "SAFE_DRIVING"
class_name["c1"] = "TEXTING_RIGHT"
class_name["c2"] = "TALKING_PHONE_RIGHT"
class_name["c3"] = "TEXTING_LEFT"
class_name["c4"] = "TALKING_PHONE_LEFT"
class_name["c5"] = "OPERATING_RADIO"
class_name["c6"] = "DRINKING"
class_name["c7"] = "REACHING_BEHIND"
class_name["c8"] = "HAIR_AND_MAKEUP"
class_name["c9"] = "TALKING_TO_PASSENGER"

print(class_name[res])

OPERATING_RADIO


In [743]:
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont

img = Image.open(img_path)
I1 = ImageDraw.Draw(img)
font = ImageFont.truetype("arial.ttf", 20)
I1.text((28, 36), class_name[res], font=font, fill=(0, 255, 0))
img.save('output_vgg/'+img_path)